# Install the relevant libraries

In [1]:
!pip install transformers wikipedia newspaper3k GoogleNews pyvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.9 MB/s 
     |████████████████████████████████| 211 kB 35.3 MB/s 
     |████████████████████████████████| 592 kB 3.7 MB/s 
     |████████████████████████████████| 163 kB 14.9 MB/s 
     |████████████████████████████████| 7.6 MB 38.0 MB/s 
     |████████████████████████████████| 93 kB 2.5 MB/s 
     |████████████████████████████████| 7.4 MB 31.2 MB/s 
     |████████████████████████████████| 81 kB 9.5 MB/s 
     |████████████████████████████████| 290 kB 49.4 MB/s 
     |████████████████████████████████| 1.6 MB 55.5 MB/s 
     |████████████████████████████████| 749 kB 59.2 MB/s 
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=e6e91770e5eaa72af566f9eae8f1cf5bf93d9f0a12fbdedeb302f50e9ee08253
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
  Crea

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import wikipedia
from newspaper import Article, ArticleException
from GoogleNews import GoogleNews
import IPython
from pyvis.network import Network

# Load the REBEL model

In [3]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

# From short text to KB

In [4]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

# Split spans: from long text to KB

In [5]:
class KB():
    def __init__(self):
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def add_relation(self, r):
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [6]:
def from_text_to_kb(text, span_length=30, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) / 
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 6
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 6,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [7]:
text = '''(9) An entity shall account for a contract with a customer that is within the scope of this Standard only when all of the following criteria are met:  the parties to the contract have approved the contract (in writing, orally or in accordance with other customary business practices) and are committed to perform their respective obligations; the entity can identify each party's rights regarding the goods or services to be transferred; the entity can identify the payment terms for the goods or services to be transferred; the contract has commercial substance (ie the risk, timing or amount of the entity's future cash flows is expected to change as a result of the contract); and it is probable that the entity will collect the consideration to which it will be entitled in exchange for the goods or services that will be transferred to the customer. In evaluating whether collectability of an amount of consideration is probable, an entity shall consider only the customer's ability and intention to pay that amount of consideration when it is due. The amount of consideration to which the entity will be entitled may be less than the price stated in the contract if the consideration is variable because the entity may offer the customer a price concession (see paragraph 52).
(22) At contract inception, an entity shall assess the goods or services promised in a contract with a customer and shall identify as a performance obligation each promise to transfer to the customer either: a good or service (or a bundle of goods or services) that is distinct; or a series of distinct goods or services that are substantially the same and that have the same pattern of transfer to the customer (see paragraph 23). 
(23) A series of distinct goods or services has the same pattern of transfer to the customer if both of the following criteria are met: each distinct good or service in the series that the entity promises to transfer to the customer would meet the criteria in paragraph 35 to be a performance obligation satisfied over time; and in accordance with paragraphs 39–40, the same method would be used to measure the entity’s progress towards complete satisfaction of the performance obligation to transfer each distinct good or service in the series to the customer.
(27) A good or service that is promised to a customer is distinct if both of the following criteria are met: the customer can benefit from the good or service either on its own or together with other resources that are readily available to the customer (ie the good or service is capable of being distinct); and the entity’s promise to transfer the good or service to the customer is separately identifiable from other promises in the contract (ie the promise to transfer the good or service is distinct within the context of the contract).
(35) An entity transfers control of a good or service over time and, therefore, satisfies a performance obligation and recognises revenue over time, if one of the following criteria is met: the customer simultaneously receives and consumes the benefits provided by the entity’s performance as the entity performs (see paragraphs B3–B4); the entity’s performance creates or enhances an asset (for example, work in progress) that the customer controls as the asset is created or enhanced (see paragraph B5); or the entity’s performance does not create an asset with an alternative use to the entity (see paragraph 36) and the entity has an enforceable right to payment for performance completed to date (see paragraph 37).'''

kb = from_text_to_kb(text, verbose=True)
kb.print()

Input has 684 tokens
Input has 23 spans
Span boundaries are [[0, 30], [29, 59], [58, 88], [87, 117], [116, 146], [145, 175], [174, 204], [203, 233], [232, 262], [261, 291], [290, 320], [319, 349], [348, 378], [377, 407], [406, 436], [435, 465], [464, 494], [493, 523], [522, 552], [551, 581], [580, 610], [609, 639], [638, 668]]
Relations:
  {'head': 'contract', 'type': 'subclass of', 'tail': 'contract', 'meta': {'spans': [[0, 30]]}}
  {'head': 'contract', 'type': 'facet of', 'tail': 'contract', 'meta': {'spans': [[0, 30], [116, 146], [522, 552]]}}
  {'head': 'contract', 'type': 'participant', 'tail': 'customer', 'meta': {'spans': [[0, 30]]}}
  {'head': 'contract', 'type': 'part of', 'tail': 'contract', 'meta': {'spans': [[0, 30], [522, 552]]}}
  {'head': 'contract', 'type': 'has part', 'tail': 'contract', 'meta': {'spans': [[0, 30], [522, 552]]}}
  {'head': 'contract', 'type': 'instance of', 'tail': 'contract', 'meta': {'spans': [[0, 30]]}}
  {'head': 'customary business practices', 'ty

# Filter and normalize entities with Wikipedia

- remove all entities that doesn't have a page on Wikipedia
- merge entities if they have the same wikipedia page

In [8]:
class KB():
    def __init__(self):
        self.entities = {}
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations
              if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"]
                        if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def get_wikipedia_data(self, candidate_entity):
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary
            }
            return entity_data
        except:
            return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

    def add_relation(self, r):
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Entities:")
        for e in self.entities.items():
            print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [9]:
text = '''(9) An entity shall account for a contract with a customer that is within the scope of this Standard only when all of the following criteria are met:  the parties to the contract have approved the contract (in writing, orally or in accordance with other customary business practices) and are committed to perform their respective obligations; the entity can identify each party's rights regarding the goods or services to be transferred; the entity can identify the payment terms for the goods or services to be transferred; the contract has commercial substance (ie the risk, timing or amount of the entity's future cash flows is expected to change as a result of the contract); and it is probable that the entity will collect the consideration to which it will be entitled in exchange for the goods or services that will be transferred to the customer. In evaluating whether collectability of an amount of consideration is probable, an entity shall consider only the customer's ability and intention to pay that amount of consideration when it is due. The amount of consideration to which the entity will be entitled may be less than the price stated in the contract if the consideration is variable because the entity may offer the customer a price concession (see paragraph 52).
(22) At contract inception, an entity shall assess the goods or services promised in a contract with a customer and shall identify as a performance obligation each promise to transfer to the customer either: a good or service (or a bundle of goods or services) that is distinct; or a series of distinct goods or services that are substantially the same and that have the same pattern of transfer to the customer (see paragraph 23). 
(23) A series of distinct goods or services has the same pattern of transfer to the customer if both of the following criteria are met: each distinct good or service in the series that the entity promises to transfer to the customer would meet the criteria in paragraph 35 to be a performance obligation satisfied over time; and in accordance with paragraphs 39–40, the same method would be used to measure the entity’s progress towards complete satisfaction of the performance obligation to transfer each distinct good or service in the series to the customer.
(27) A good or service that is promised to a customer is distinct if both of the following criteria are met: the customer can benefit from the good or service either on its own or together with other resources that are readily available to the customer (ie the good or service is capable of being distinct); and the entity’s promise to transfer the good or service to the customer is separately identifiable from other promises in the contract (ie the promise to transfer the good or service is distinct within the context of the contract).
(35) An entity transfers control of a good or service over time and, therefore, satisfies a performance obligation and recognises revenue over time, if one of the following criteria is met: the customer simultaneously receives and consumes the benefits provided by the entity’s performance as the entity performs (see paragraphs B3–B4); the entity’s performance creates or enhances an asset (for example, work in progress) that the customer controls as the asset is created or enhanced (see paragraph B5); or the entity’s performance does not create an asset with an alternative use to the entity (see paragraph 36) and the entity has an enforceable right to payment for performance completed to date (see paragraph 37).'''
kb = from_text_to_kb(text)
kb.print()
# Entities:
#  ('Napoleon', {'url': 'https://en.wikipedia.org/wiki/Napoleon',
#   'summary': "Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August ..."})
#  ('French Revolution', {'url': 'https://en.wikipedia.org/wiki/French_Revolution',
#   'summary': 'The French Revolution (French: Révolution française..."})
#  ...
# Relations:
#  {'head': 'Napoleon', 'type': 'participant in', 'tail': 'French Revolution',
#   'meta': {'spans': [[0, 128], [119, 247]]}}
#  {'head': 'French Revolution', 'type': 'participant', 'tail': 'Napoleon',
#   'meta': {'spans': [[0, 128]]}}
#  ...

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Entities:
  ('Contract', {'url': 'https://en.wikipedia.org/wiki/Contract', 'summary': 'A contract is a legally enforceable agreement that creates, defines, and governs mutual rights and obligations among its parties. A contract typically involves the transfer of goods, services, money, or a promise to transfer any of those at a future date. In the event of a breach of contract, the injured party may seek judicial remedies such as damages or rescission. Contract law, the field of the law of obligations concerned with contracts, is based on the principle that agreements must be honoured.Contract law, like other areas of private law, varies between jurisdictions. The various systems of contract law can broadly be split between common law jurisdictions, civil law jurisdictions, and mixed law jurisdictions which combine elements of both common and civil law. Common law jurisdictions typically require contracts to include consideration in order to be valid, whereas civil and most mixed law j

# Extract KB from web article

In [10]:
def from_text_to_kb(text, article_url, span_length=128, article_title=None,
                    article_publish_date=None, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) / 
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                article_url: {
                    "spans": [spans_boundaries[current_span_index]]
                }
            }
            kb.add_relation(relation, article_title, article_publish_date)
        i += 1

    return kb

In [11]:
class KB():
    def __init__(self):
        self.entities = {} # { entity_title: {...} }
        self.relations = [] # [ head: entity_title, type: ..., tail: entity_title,
          # meta: { article_url: { spans: [...] } } ]
        self.sources = {} # { article_url: {...} }

    def merge_with_kb(self, kb2):
        for r in kb2.relations:
            article_url = list(r["meta"].keys())[0]
            source_data = kb2.sources[article_url]
            self.add_relation(r, source_data["article_title"],
                              source_data["article_publish_date"])

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r2):
        r1 = [r for r in self.relations
              if self.are_relations_equal(r2, r)][0]

        # if different article
        article_url = list(r2["meta"].keys())[0]
        if article_url not in r1["meta"]:
            r1["meta"][article_url] = r2["meta"][article_url]

        # if existing article
        else:
            spans_to_add = [span for span in r2["meta"][article_url]["spans"]
                            if span not in r1["meta"][article_url]["spans"]]
            r1["meta"][article_url]["spans"] += spans_to_add

    def get_wikipedia_data(self, candidate_entity):
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary
            }
            return entity_data
        except:
            return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

    def add_relation(self, r, article_title, article_publish_date):
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # add source if not in kb
        article_url = list(r["meta"].keys())[0]
        if article_url not in self.sources:
            self.sources[article_url] = {
                "article_title": article_title,
                "article_publish_date": article_publish_date
            }

        # manage new relation
        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    def print(self):
        print("Entities:")
        for e in self.entities.items():
            print(f"  {e}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")
        print("Sources:")
        for s in self.sources.items():
            print(f"  {s}")

In [12]:
def get_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article

def from_url_to_kb(url):
    article = get_article(url)
    config = {
        "article_title": article.title,
        "article_publish_date": article.publish_date
    }
    kb = from_text_to_kb(article.text, article.url, **config)
    return kb

In [13]:
url = "https://finance.yahoo.com/news/microstrategy-bitcoin-millions-142143795.html"
kb = from_url_to_kb(url)
kb.print()
# Entities:
#   ('MicroStrategy', {'url': 'https://en.wikipedia.org/wiki/MicroStrategy',
#     'summary': "MicroStrategy Incorporated is an American company that ..."})
#   ('Michael J. Saylor', {'url': 'https://en.wikipedia.org/wiki/Michael_J._Saylor',
#     'summary': 'Michael J. Saylor (born February 4, 1965) is an American ..."})
#   ...
# Relations:
#   {'head': 'MicroStrategy', 'type': 'founded by', 'tail': 'Michael J. Saylor',
#    'meta': {'https://finance.yahoo.com/news/microstrategy-bitcoin-millions-142143795.html': 
#      {'spans': [[0, 128]]}}}
#   {'head': 'Michael J. Saylor', 'type': 'employer', 'tail': 'MicroStrategy',
#    'meta': {'https://finance.yahoo.com/news/microstrategy-bitcoin-millions-142143795.html':
#      {'spans': [[0, 128]]}}}
#   ...
# Sources:
#   ('https://finance.yahoo.com/news/microstrategy-bitcoin-millions-142143795.html',
#     {'article_title': "Microstrategy chief: 'Bitcoin is going to go into the millions'",
#      'article_publish_date': None})

Entities:
  ('MicroStrategy', {'url': 'https://en.wikipedia.org/wiki/MicroStrategy', 'summary': "MicroStrategy Incorporated is an American company that provides business intelligence (BI), mobile software, and cloud-based services.\nFounded in 1989 by Michael J. Saylor, Sanju Bansal, and Thomas Spahr, the firm develops software to analyze internal and external data in order to make business decisions and to develop mobile apps. It is a public company headquartered in Tysons Corner, Virginia, in the Washington metropolitan area. Its primary business analytics competitors include SAP AG Business Objects, IBM Cognos, and Oracle Corporation's BI Platform. Saylor is the Executive Chairman and, from 1989 to 2022, was the CEO.\n\n"})
  ('Michael J. Saylor', {'url': 'https://en.wikipedia.org/wiki/Michael_J._Saylor', 'summary': "Michael J. Saylor (born February 4, 1965) is an American entrepreneur and business executive. He is the executive chairman and a co-founder of MicroStrategy, a company 

# Google News: extract KB from multiple articles

In [14]:
def get_news_links(query, lang="en", region="US", pages=1, max_links=100000):
    googlenews = GoogleNews(lang=lang, region=region)
    googlenews.search(query)
    all_urls = []
    for page in range(pages):
        googlenews.get_page(page)
        all_urls += googlenews.get_links()
    return list(set(all_urls))[:max_links]

def from_urls_to_kb(urls, verbose=False):
    kb = KB()
    if verbose:
        print(f"{len(urls)} links to visit")
    for url in urls:
        if verbose:
            print(f"Visiting {url}...")
        try:
            kb_url = from_url_to_kb(url)
            kb.merge_with_kb(kb_url)
        except ArticleException:
            if verbose:
                print(f"  Couldn't download article at url {url}")
    return kb

In [15]:
import pickle

def save_kb(kb, filename):
    with open(filename, "wb") as f:
        pickle.dump(kb, f)

def load_kb(filename):
    res = None
    with open(filename, "rb") as f:
        res = pickle.load(f)
    return res

In [16]:
news_links = get_news_links("Google", pages=1, max_links=3)
kb = from_urls_to_kb(news_links, verbose=True)
kb.print()
# 3 links to visit
# Visiting https://www.hindustantimes.com/india-news/google-doodle-celebrates-india-s-gama-pehlwan-the-undefeated-wrestling-champion-101653180853982.html...
# Visiting https://tech.hindustantimes.com/tech/news/google-doodle-today-celebrates-gama-pehlwan-s-144th-birth-anniversary-know-who-he-is-71653191916538.html...
# Visiting https://www.moneycontrol.com/news/trends/current-affairs-trends/google-doodle-celebrates-gama-pehlwan-the-amritsar-born-wrestling-champ-who-inspired-bruce-lee-8552171.html...
# Entities:
#   ('Google', {'url': 'https://en.wikipedia.org/wiki/Google',
#     'summary': 'Google LLC is an American ...'})
#   ...
# Relations:
#   {'head': 'Google', 'type': 'owner of', 'tail': 'Google Doodle',
#     'meta': {'https://tech.hindustantimes.com/tech/news/google-doodle-today-celebrates-gama-pehlwan-s-144th-birth-anniversary-know-who-he-is-71653191916538.html':
#       {'spans': [[0, 128]]}}}
#   ...
# Sources:
#   ('https://www.hindustantimes.com/india-news/google-doodle-celebrates-india-s-gama-pehlwan-the-undefeated-wrestling-champion-101653180853982.html',
#     {'article_title': "Google Doodle celebrates India's Gama Pehlwan, the undefeated wrestling champion",
#     'article_publish_date': datetime.datetime(2022, 5, 22, 6, 59, 56, tzinfo=tzoffset(None, 19800))})
#   ('https://tech.hindustantimes.com/tech/news/google-doodle-today-celebrates-gama-pehlwan-s-144th-birth-anniversary-know-who-he-is-71653191916538.html',
#     {'article_title': "Google Doodle today celebrates Gama Pehlwan's 144th birth anniversary; know who he is",
#     'article_publish_date': datetime.datetime(2022, 5, 22, 9, 32, 38, tzinfo=tzoffset(None, 19800))})
#   ('https://www.moneycontrol.com/news/trends/current-affairs-trends/google-doodle-celebrates-gama-pehlwan-the-amritsar-born-wrestling-champ-who-inspired-bruce-lee-8552171.html',
#     {'article_title': 'Google Doodle celebrates Gama Pehlwan, the Amritsar-born wrestling champ who inspired Bruce Lee',
#     'article_publish_date': None})

'NoneType' object has no attribute 'group'
3 links to visit
Visiting https://9to5google.com/2022/10/31/pixel-7-android-auto-fix/...
Visiting https://www.wsj.com/articles/why-google-plays-down-its-ad-tech-business-but-is-determined-to-keep-it-11667292084...
  Couldn't download article at url https://www.wsj.com/articles/why-google-plays-down-its-ad-tech-business-but-is-determined-to-keep-it-11667292084
Visiting https://www.searchenginejournal.com/google-introduces-new-search-labels-for-coupons-promos/469828/...
Entities:
  ('Pixel 7', {'url': 'https://en.wikipedia.org/wiki/Pixel_7', 'summary': 'The Pixel 7 and Pixel 7 Pro are a pair of Android smartphones designed, developed, and marketed by Google as part of the Google Pixel product line. They serve as the successor to the Pixel 6 and Pixel 6 Pro, respectively. The phones were first previewed in May 2022, during the Google I/O keynote. They are powered by the second-generation Google Tensor chip, and features a design similar to that o

# Visualize KB

In [17]:
def save_network_html(kb, filename="network.html"):
    # create network
    net = Network(directed=True, width="700px", height="700px", bgcolor="#eeeeee")

    # nodes
    color_entity = "#00FF00"
    for e in kb.entities:
        net.add_node(e, shape="circle", color=color_entity)

    # edges
    for r in kb.relations:
        net.add_edge(r["head"], r["tail"],
                    title=r["type"], label=r["type"])
        
    # save network
    net.repulsion(
        node_distance=200,
        central_gravity=0.2,
        spring_length=200,
        spring_strength=0.05,
        damping=0.09
    )
    net.set_edge_smooth('dynamic')
    net.show(filename)

In [18]:
news_links = get_news_links("Google", pages=5, max_links=20)
kb = from_urls_to_kb(news_links, verbose=True)
print(kb)
filename = "network_3_google.html"
filename = "test.html"
save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

20 links to visit
Visiting https://techcrunch.com/2022/10/27/google-cloud-gets-into-web3-act-with-managed-blockchain-node-service/...
Visiting https://www.searchenginejournal.com/google-introduces-new-search-labels-for-coupons-promos/469828/...
Visiting https://www.androidpolice.com/google-pixel-7-review/...
  Couldn't download article at url https://www.androidpolice.com/google-pixel-7-review/
Visiting https://www.cnbc.com/2022/10/28/why-investors-rewarded-apple-but-fled-amazon-google-facebook-after-earnings.html...
Visiting https://techcrunch.com/2022/10/31/google-pauses-enforcement-play-store-billing-requirement-in-india-following-antitrust-order/...
Visiting https://www.reuters.com/technology/southeast-asia-internet-economy-forecast-cut-economic-headwinds-2022-10-27/...
Visiting https://www.forbes.com/sites/johanmoreno/2022/10/31/study-57-of-smartphone-users-dont-click-on-google-results/...
  Couldn't download article at url https://www.forbes.com/sites/johanmoreno/2022/10/31/study

Token indices sequence length is longer than the specified maximum sequence length for this model (1614 > 1024). Running this sequence through the model will result in indexing errors


Visiting https://www.wired.com/story/best-google-pixel-phone/...
Visiting https://www.prnewswire.com/news-releases/micro-focus-announces-integration-with-google-cloud-dual-run-for-mainframe-modernization-301660849.html...
Visiting https://nextcity.org/features/google-sidewalk-labs-what-makes-a-city-detroit-toronto...
Visiting https://www.propublica.org/article/google-alphabet-ads-fund-disinformation-covid-elections...
Visiting https://www.techtarget.com/searchbusinessanalytics/news/252526597/ThoughtSpot-launches-integration-with-Google-Sheets...
Visiting https://techcrunch.com/2022/10/27/google-acquires-twitter-backed-ai-avatar-startup-alter-for-100-million-source-says/...


In [19]:
#news_links = get_news_links("Google", pages=5, max_links=20)
#kb = from_urls_to_kb(news_links, verbose=True)
filename = "network_3_google.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

In [ ]:
news_links = get_news_links("Amazon", pages=5, max_links=20)
kb = from_urls_to_kb(news_links, verbose=True)
filename = "network_3_amazon.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

20 links to visit
Visiting https://www.yahoo.com/lifestyle/amazon-basics-dutch-oven-194154613.html...
Visiting https://screenrant.com/prime-members-access-amazon-music-entire-catalog-catch/...
  Couldn't download article at url https://screenrant.com/prime-members-access-amazon-music-entire-catalog-catch/
Visiting https://finance.yahoo.com/video/paramount-set-report-q3-earnings-204420731.html...
Visiting https://www.msn.com/en-us/lifestyle/shopping/olaplex-revlon-and-more-are-on-sale-on-amazon-right-now/ar-AA13CS25...
Visiting https://www.digitalmusicnews.com/2022/11/01/amazon-music-full-catalog-prime-members/...
Visiting https://www.yakimaherald.com/ap/business/amazon-music/image_9fd614f5-79bb-5dc7-bb26-a390c9b1a493.html...
Visiting https://www.hackread.com/amazon-prime-video-viewing-habits/...
Visiting https://www.techradar.com/news/amazon-to-make-its-full-music-streaming-library-free-to-prime-members-but-theres-a-catch...
Visiting https://mynbc15.com/features/interviews/have-your-te

In [ ]:
news_links = get_news_links("Apple", pages=5, max_links=20)
kb = from_urls_to_kb(news_links, verbose=True)
filename = "network_3_apple.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

'NoneType' object has no attribute 'group'
20 links to visit
Visiting https://www.cnbc.com/2022/11/01/apple-iphone-maker-foxconn-says-no-covid-deaths-at-china-plant.html...
Visiting https://www.gizmochina.com/2022/11/01/apple-iphone-se-4s-display-size-yet-to-be-confirmed-oled-panel-in-rumors/...
  Couldn't download article at url https://www.gizmochina.com/2022/11/01/apple-iphone-se-4s-display-size-yet-to-be-confirmed-oled-panel-in-rumors/
Visiting https://macdailynews.com/2022/11/01/apple-supplier-foxconn-quadruples-bonuses-to-staff-hit-by-yet-another-china-covid-lockdown/...
Visiting https://seekingalpha.com/news/3898675-apple-gets-nipped-on-report-of-app-store-sales-decline...
Visiting https://9to5mac.com/2022/11/01/eu-force-third-party-app-stores-on-apple/...
Visiting https://www.fool.com/investing/2022/11/01/apple-bank-of-america-and-chevron-warren-buffett/...
Visiting https://www.investors.com/how-to-invest/investors-corner/tech-stocks-a-buy-what-to-watch-in-apple-microsoft-intel

In [ ]:
news_links = get_news_links("Elon Musk", pages=5, max_links=20)
kb = from_urls_to_kb(news_links, verbose=True)
filename = "network_3_musk.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

20 links to visit
Visiting https://www.youtube.com/watch?v=mU87pBMjqPc...
Visiting https://www.vox.com/recode/2022/11/1/23435301/elon-musk-twitter-deal-winning-losing-conservatives-free-speech-trump-jason-calacanis...
Visiting https://www.theguardian.com/technology/2022/nov/01/musk-charging-twitter-verified-accounts...
Visiting https://metro.co.uk/2022/11/01/elon-musk-unveils-new-8-a-month-twitter-blue-checkmark-charge-17680126/...
Visiting https://www.dailymail.co.uk/news/article-11378569/Elon-Musk-confirms-blue-tick-cost-8-month-bulls.html...
Visiting https://www.nationalreview.com/news/elon-musk-announces-plan-to-overhaul-twitters-lords-and-peasants-verification-process/...
Visiting https://timesofindia.indiatimes.com/business/international-business/blue-tick-verified-twitter-accounts-to-cost-8-per-month-elon-musk/articleshow/95238517.cms...
Visiting https://www.cnn.com/2022/11/01/tech/musk-twitter-verification-price/index.html...
Visiting https://www.nytimes.com/2022/11/01/technolo

In [ ]:
news_links = get_news_links("Kobe Bryant", pages=5, max_links=20)
kb = from_urls_to_kb(news_links, verbose=True)
filename = "network_3_bryant.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

20 links to visit
Visiting https://www.talkbasket.net/153363-the-best-nba-players-of-all-time...
Visiting https://apnews.com/article/kobe-bryant-denver-nuggets-los-angeles-lakers-nba-sports-d392d0aa5d50fd39bd50db87e787f8f6...
Visiting https://thesportsrush.com/nba-news-lebron-james-michael-jordan-kobe-bryant-theyre-all-the-same-tyronn-lue-lists-similarities-between-the-three-greats/...
Visiting https://www.essentiallysports.com/boxing-news-nba-youre-an-ashole-mike-tyson-threatens-to-beat-up-kobe-bryant-lookalike-for-this-emmy-nominated-actress-on-hotboxin/...
Visiting https://www.talkbasket.net/153357-dwight-howard-on-kobe-bryant-calling-him-soft...
Visiting https://www.inc.com/jeff-haden/a-shrewd-leadership-lesson-from-kobe-bryantnetflix-documentary.html...
  Couldn't download article at url https://www.inc.com/jeff-haden/a-shrewd-leadership-lesson-from-kobe-bryantnetflix-documentary.html
Visiting https://people.com/sports/lamar-odom-fan-gifted-nba-championship-rings-he-pawned/...
Vis

In [ ]:
text = "Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk."
kb = from_text_to_kb(text, "", verbose=True)
filename = "network_1_napoleon.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

Input has 207 tokens
Input has 2 spans
Span boundaries are [[0, 128], [79, 207]]


In [ ]:
text = "Kobe Bean Bryant (August 23, 1978 – January 26, 2020) was an American professional basketball player. A shooting guard, he spent his entire 20-year career with the Los Angeles Lakers in the National Basketball Association (NBA). Widely regarded as one of the greatest basketball players of all time, Bryant won five NBA championships, was an 18-time All-Star, a 15-time member of the All-NBA Team, a 12-time member of the All-Defensive Team, the 2008 NBA Most Valuable Player (MVP), and a two-time NBA Finals MVP. Bryant also led the NBA in scoring twice, and ranks fourth in league all-time regular season and postseason scoring. He was posthumously voted into the Naismith Memorial Basketball Hall of Fame in 2020 and named to the NBA 75th Anniversary Team in 2021."
kb = from_text_to_kb(text, "", verbose=True)
filename = "network_1_bryant.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

Input has 176 tokens
Input has 2 spans
Span boundaries are [[0, 128], [48, 176]]


In [ ]:
text = '''Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk.'''
kb = from_text_to_kb(text, "", verbose=True)
filename = "network_1_google.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

Input has 207 tokens
Input has 2 spans
Span boundaries are [[0, 128], [79, 207]]


In [ ]:
url = "https://www.investopedia.com/terms/c/cryptocurrency.asp"
kb = from_url_to_kb(url)
filename = "network_2_crypto.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

In [ ]:
url = "https://www.britannica.com/biography/Johnny-Depp"
kb = from_url_to_kb(url)
filename = "network_2_depp.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

In [ ]:
url = "https://www.timeout.com/rome/things-to-do/best-things-to-do-in-rome"
kb = from_url_to_kb(url)
filename = "network_2_rome.html"
save_network_html(kb, filename=filename)
save_kb(kb, filename.split(".")[0] + ".p")
IPython.display.HTML(filename=filename)

In [ ]:
!pip install neuralcoref

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 286 kB 7.8 MB/s 
     |████████████████████████████████| 132 kB 47.9 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 9.8 MB 50.8 MB/s 
     |████████████████████████████████| 127 kB 48.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install spacy==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 27.7 MB 26.9 MB/s 
     |████████████████████████████████| 3.2 MB 27.3 MB/s 
     |████████████████████████████████| 82 kB 249 kB/s 
     |████████████████████████████████| 2.1 MB 46.7 MB/s 
     |████████████████████████████████| 208 kB 52.8 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.8
    Uninstalling preshed-3.0.8:
      Successfully uninstalled preshed-3.0.8
  Attempting uninstall: blis
    Found existing installation: blis 0.7.9
    Uninstalling blis-0.7.9:
      Successfully uninstalled blis-0.7.9
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uni

In [ ]:
!python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.1 MB 7.4 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074433 sha256=837d2d361d6917f0394505eefc7b23d7a1e7aebbb7d43f7dccfdebeb86a40fe8
  Stored in directory: /tmp/pip-ephem-wheel-cache-8bq2dycn/wheels/59/4f/8c/0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.1
    Uninstalling en-core-web-sm-3.4.1:
      Successfully uninstalled en-core-web-sm-3.4.1
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import spacy
import neuralcoref

# Load SpaCy
nlp = spacy.load('en')
# Add neural coref to SpaCy's pipe
neuralcoref.add_to_pipe(nlp)

def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

    return "".join(tok_list)

ImportError: ignored

In [ ]:
import urllib
from string import punctuation
import nltk

ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

def wikifier(text, lang="en", threshold=0.8):
    """Function that fetches entity linking results from wikifier.com API"""
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "tgbdmkpmkluegqfbawcwjywieevmza"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout=60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.
    results = list()
    for annotation in response["annotations"]:
        # Filter out desired entity classes
        if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):

            # Specify entity label
            if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
                label = 'Person'
            elif any([el['enLabel'] in ["company", "enterprise", "business", "organization"] for el in annotation['wikiDataClasses']]):
                label = 'Organization'
            elif any([el['enLabel'] in ["geographic region", "human settlement", "geographic entity", "territorial entity type"] for el in annotation['wikiDataClasses']]):
                label = 'Location'
            else:
                label = None

            results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'], 'label': label,
                            'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    return results

In [ ]:
sentence='''Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk.'''

In [ ]:
# First get all the entities in the sentence
entities = wikifier(sentence, threshold=entities_threshold)
# Iterate over every permutation pair of entities
for permutation in itertools.permutations(entities, 2):
    for source in permutation[0]['characters']:
        for target in permutation[1]['characters']:
            # Relationship extraction with OpenNRE
            data = relation_model.infer(
                {'text': sentence, 'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})
            if data[1] > relation_threshold:
                relations_list.append(
                    {'source': permutation[0]['title'], 'target': permutation[1]['title'], 'type': data[0]})